In [2]:
# 📦 Install required packages
!pip install numpy==1.26.4 scikit-surprise --quiet

# ✅ Imports
import pandas as pd
import json
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import joblib

# ✅ Step 1: Load and prepare data
df = pd.read_csv('Walmart_customer_purchases.csv')  # Replace with your actual path

# Use only relevant columns and drop missing ratings
df = df[['Customer_ID', 'Product_Name', 'Rating']].dropna()
df['Customer_ID'] = df['Customer_ID'].astype(str)
df['Product_Name'] = df['Product_Name'].astype(str)

# ✅ Step 2: Create Surprise dataset
reader = Reader(rating_scale=(1, 5))  # or adjust if your ratings vary
data = Dataset.load_from_df(df[['Customer_ID', 'Product_Name', 'Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# ✅ Step 3: Train model
model = SVD()
model.fit(trainset)

# ✅ Step 4: Get sample customers and products
sample_customers = df['Customer_ID'].unique()[:100]
sample_products = df['Product_Name'].unique()[:1000]

# ✅ Step 5: Cache known products for each user
user_seen = df.groupby('Customer_ID')['Product_Name'].apply(set).to_dict()

# ✅ Step 6: Recommend top 5 products for each customer
recommendation_dict = {}

for customer in sample_customers:
    seen = user_seen.get(customer, set())
    unseen = [p for p in sample_products if p not in seen]

    predictions = [model.predict(customer, p) for p in unseen]
    top_preds = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

    recommendation_dict[customer] = [pred.iid for pred in top_preds]

# ✅ Step 7: Save to JSON for Flutter app
with open('recommendations.json', 'w') as f:
    json.dump(recommendation_dict, f, indent=4)

print("✅ Top 5 product recommendations saved to 'recommendations.json'")


✅ Top 5 product recommendations saved to 'recommendations.json'


In [3]:
#  export this model for testing

joblib.dump(model, 'svd_model.pkl')
print("✅ Trained SVD model saved as 'svd_model.pkl'")


✅ Trained SVD model saved as 'svd_model.pkl'
